In [1]:
epoche = 5

# Algoritmo originale

In [ ]:
import argparse
import os
import pickle
from pennylane import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import pandas as pd
import json

def get_args():
    parser = argparse.ArgumentParser(
        description="Parse the script arguments."
    )

    parser.add_argument(
        "--dataset-path",
        type=str,
        required=True,
        help="Path to the dataset."
    )

    parser.add_argument(
        "--output-dir",
        type=str,
        required=True,
        help="Path to the output directory."
    )

    return parser.parse_args()

def ingest_dataset(path, n_pkts=10, n_features=4):
    with open(path, "rb") as f:
        biflows = pickle.load(f)
        labels = pickle.load(f)
    biflows = np.array(biflows)[:,:n_pkts,:n_features]
    return biflows, labels

if __name__ == "__main__":
    # Parsing degli argomenti
    args = get_args()
    dataset_path = args.dataset_path
    output_dir = args.output_dir

    n_pkts = 10
    n_features = 4
    seed = 2025
    
    n_qubits = 5
    n_layers = 3

    # Riproducibilità
    np.random.seed(seed)                    # NumPy
    tf.random.set_seed(seed)                # TensorFlow
    tf.keras.utils.set_random_seed(seed)    # Keras

    # Caricamento del dataset
    X, y = ingest_dataset(dataset_path, n_pkts=n_pkts, n_features=n_features)
    num_classes = len(np.unique(y))

    # Codifica delle label
    le = LabelEncoder()
    y = le.fit_transform(y)

    # Partizionamento in train, validation e test set (proporzioni 80/20, 80/20)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y,
    )

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(np.reshape(X_train, [-1, n_features]))
    res_samples_train = scaler.transform(np.reshape(X_train, [-1, n_features]))
    res_samples_test = scaler.transform(np.reshape(X_test, [-1, n_features]))
    X_train = np.reshape(res_samples_train, [-1, n_pkts, n_features])
    X_test = np.reshape(res_samples_test, [-1, n_pkts, n_features])

    X_train, X_valid, y_train, y_valid = train_test_split(
        X_train, y_train, test_size=0.2, random_state=seed, stratify=y_train,
    )

    ohe_y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    ohe_y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=num_classes)

    # Definizione del circuito quantistico
    dev = qml.device("default.qubit", seed=seed, wires=n_qubits)
    @qml.qnode(dev , interface="tf")
    def qnode(inputs, weights):
        
        # Feature map
        qml.AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True, pad_with=0.0)

        # Ansatz
        qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))

        # Processo di misurazione
        return qml.probs(wires=range(n_qubits))

    # Pesi dell'ansatz
    weights = np.random.rand(n_layers*n_qubits*3).reshape(n_layers,n_qubits,3)
    weights = tf.Variable(weights, dtype=tf.float64, trainable=True)
    weight_shapes = {"weights": (n_layers,n_qubits,3)}

    #Definizione del modello
    model = Sequential(name='ampe_fixed_probs')
    model.add(Flatten())
    model.add(Dense(2**n_qubits,activation='sigmoid', input_dim=n_pkts*n_features))
    model.add(qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2**n_qubits)) # <-- qui c'è la parte quantum
    model.add(Dense(num_classes, activation='softmax'))
    model.build(np.shape(X_train))

    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')
    callbacks = [earlystop]

    os.makedirs(f"{output_dir}", exist_ok=True)

    with open(f"{output_dir}/model_summary.txt", "w") as f:
        model.summary(print_fn=lambda x: f.write(x + "\n"))

    history = model.fit(X_train, ohe_y_train, validation_data=(X_valid,ohe_y_valid),
                    epochs=5, batch_size=50,
                    callbacks=callbacks, verbose=2)
    
    os.makedirs(output_dir, exist_ok=True)
    df_history = pd.DataFrame(history.history)
    df_history.to_csv(f"{output_dir}/training_history.csv", index=False)

    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Salvataggio dei risultati
    soft_values = [",".join(map(str, probs)) for probs in y_pred_probs]
    df_soft = pd.DataFrame({
        "Actual": y_test,
        "soft_values": soft_values
    })
    df_pred = pd.DataFrame({
        "Actual": y_test,
        "Predicted": y_pred
    })

    df_soft.to_csv(f"{output_dir}/soft_values.dat", sep="\t", index=False)
    df_pred.to_csv(f"{output_dir}/predictions.dat", sep="\t", index=False)
    labels_map = {}
    for c, enc_c in zip(le.classes_, le.transform(le.classes_)):
        labels_map[str(enc_c)] = c
    with open(f"{output_dir}/labels_map.json", 'w') as f:
        json.dump(labels_map, f)

In [ ]:
import argparse
import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import pandas as pd
import json

for i in range(0,2):
    seed = 2025 + i
    def ingest_dataset(path, n_pkts=10, n_features=4):
        """Carica dataset da pickle e seleziona i primi n_pkts e n_features."""
        with open(path, "rb") as f:
            biflows = pickle.load(f)
            labels = pickle.load(f)
        biflows = np.array(biflows)[:, :n_pkts, :n_features]
        return biflows, labels

    if __name__ == "__main__":

        dataset_path = "mirage2019_LOPEZ_lopez_lopez_36P_4F_APP_xST_PAD_metadata.pickle"
        output_dir = "output5/"
        os.makedirs(output_dir, exist_ok=True)

        n_pkts = 10
        n_features = 4
        sample_train_size = 50000
        epoche = epoche
        seed = seed

        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

        X, y = ingest_dataset(dataset_path, n_pkts=n_pkts, n_features=n_features)

        le = LabelEncoder()
        y = le.fit_transform(y)
        num_classes = len(np.unique(y))

        X_train_full, X_test, y_train_full, y_test = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=seed
        )

        if sample_train_size < len(X_train_full):
            X_train_full, _, y_train_full, _ = train_test_split(
                X_train_full, y_train_full,
                train_size=sample_train_size,
                stratify=y_train_full,
                random_state=seed
            )

        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train_full_scaled = scaler.fit_transform(X_train_full.reshape(-1, n_features)).reshape(-1, n_pkts, n_features)
        X_test_scaled = scaler.transform(X_test.reshape(-1, n_features)).reshape(-1, n_pkts, n_features)

        X_train, X_valid, y_train, y_valid = train_test_split(
            X_train_full_scaled, y_train_full,
            test_size=0.2, stratify=y_train_full, random_state=seed
        )

        ohe_y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
        ohe_y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=num_classes)

        model = Sequential([
            Conv2D(32, (4, 2), padding='same', activation='relu', input_shape=(n_pkts, n_features, 1)),
            MaxPooling2D((3, 2), padding='same'),
            BatchNormalization(),
            Conv2D(64, (4, 2), padding='same', activation='relu'),
            MaxPooling2D((3, 1), padding='same'),
            BatchNormalization(),
            Flatten(),
            Dense(200, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        earlystop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

        with open(f"{output_dir}/model_summary.txt", "w") as f:
            model.summary(print_fn=lambda x: f.write(x + "\n"))

        X_train_cnn = X_train.reshape(-1, n_pkts, n_features, 1)
        X_valid_cnn = X_valid.reshape(-1, n_pkts, n_features, 1)
        X_test_cnn  = X_test_scaled.reshape(-1, n_pkts, n_features, 1)

        history = model.fit(
            X_train_cnn, ohe_y_train,
            validation_data=(X_valid_cnn, ohe_y_valid),
            epochs=epoche,
            batch_size=50,
            callbacks=[earlystop],
            verbose=2
        )

        pd.DataFrame(history.history).to_csv(f"{output_dir}/training_history{i}.csv", index=False)

        y_pred_probs = model.predict(X_test_cnn)
        y_pred = np.argmax(y_pred_probs, axis=1)

        df_soft = pd.DataFrame({
            "Actual": y_test,
            "soft_values": [",".join(map(str, p)) for p in y_pred_probs]
        })
        df_soft.to_csv(f"{output_dir}/soft_values{i}.dat", sep="\t", index=False)

        df_pred = pd.DataFrame({
            "Actual": y_test,
            "Predicted": y_pred
        })
        df_pred.to_csv(f"{output_dir}/predictions{i}.dat", sep="\t", index=False)

        labels_map = {str(enc): cls for cls, enc in zip(le.classes_, le.transform(le.classes_))}
        with open(f"{output_dir}/labels_map{i}.json", "w") as f:
            json.dump(labels_map, f)

        print("✓ Training completato e file salvati.")


In [6]:
import os
import pickle
from pennylane import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import json

# ===============================
# Funzione caricamento dataset
# ===============================
def ingest_dataset(path, n_pkts=10, n_features=4):
    with open(path, "rb") as f:
        biflows = pickle.load(f)
        labels = pickle.load(f)
    biflows = np.array(biflows)[:, :n_pkts, :n_features]
    return biflows, labels


if __name__ == "__main__":

    # ===============================
    # PATH COME NEL CNN MODEL
    # ===============================
    dataset_path = "../mirage2019_LOPEZ_lopez_lopez_36P_4F_APP_xST_PAD_metadata.pickle"
    output_dir = "quantum/output"
    os.makedirs(output_dir, exist_ok=True)

    # Parametri dataset
    n_pkts = 10
    n_features = 4
    sample_train_size = 10000
    epoche = 5

    # Quantum params
    n_qubits = 5
    n_layers = 3

    # Seed globale (non modificato)
    seed = 42
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.utils.set_random_seed(seed)

    # ===============================
    # CARICAMENTO
    # ===============================
    X, y = ingest_dataset(dataset_path, n_pkts=n_pkts, n_features=n_features)

    le = LabelEncoder()
    y = le.fit_transform(y)
    num_classes = len(np.unique(y))

    # ===============================
    # TRAIN / TEST
    # ===============================
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=seed
    )

    # Sampling
    if sample_train_size < len(X_train_full):
        X_train_full, _, y_train_full, _ = train_test_split(
            X_train_full, y_train_full,
            train_size=sample_train_size,
            stratify=y_train_full,
            random_state=seed
        )

    # ===============================
    # NORMALIZZAZIONE
    # ===============================
    scaler = MinMaxScaler((0,1))
    X_train_full_scaled = scaler.fit_transform(
        X_train_full.reshape(-1, n_features)
    ).reshape(-1, n_pkts, n_features)

    X_test_scaled = scaler.transform(
        X_test.reshape(-1, n_features)
    ).reshape(-1, n_pkts, n_features)

    # ===============================
    # TRAIN / VALID
    # ===============================
    X_train, X_valid, y_train, y_valid = train_test_split(
        X_train_full_scaled, y_train_full,
        test_size=0.2, stratify=y_train_full, random_state=seed
    )

    ohe_y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    ohe_y_valid = tf.keras.utils.to_categorical(y_valid, num_classes)

    # ===============================
    # QUANTUM DEVICE
    # ===============================
    dev = qml.device("default.qubit", wires=n_qubits, seed=seed)

    @qml.qnode(dev, interface="tf")
    def qnode(inputs, weights):
        qml.AmplitudeEmbedding(inputs, wires=range(n_qubits),
                               normalize=True, pad_with=0.0)
        qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        return qml.probs(wires=range(n_qubits))

    weight_shapes = {"weights": (n_layers, n_qubits, 3)}

    # ===============================
    # MODELLO QUANTISTICO
    # ===============================
    model = Sequential(name="qml_stratified")
    model.add(Flatten(input_shape=(n_pkts, n_features)))
    model.add(Dense(2**n_qubits, activation='sigmoid'))
    model.add(qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2**n_qubits))
    model.add(Dense(num_classes, activation="softmax"))

    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    
    model.compile(
        optimizer=Adam(0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    earlystop = EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # ===============================
    # TRAINING
    # ===============================
    history = model.fit(
        X_train, ohe_y_train,
        validation_data=(X_valid, ohe_y_valid),
        epochs=epoche,
        batch_size=50,
        callbacks=[earlystop],
        verbose=2
    )

    # ===============================
    # SALVATAGGIO RISULTATI
    # ===============================
    pd.DataFrame(history.history).to_csv(f"{output_dir}/training_history.csv", index=False)

    y_pred_probs = model.predict(X_test_scaled)
    y_pred = np.argmax(y_pred_probs, axis=1)

    df_soft = pd.DataFrame({
        "Actual": y_test,
        "soft_values": [",".join(map(str, p)) for p in y_pred_probs]
    })
    df_soft.to_csv(f"{output_dir}/soft_values.dat", sep="\t", index=False)

    df_pred = pd.DataFrame({
        "Actual": y_test,
        "Predicted": y_pred
    })
    df_pred.to_csv(f"{output_dir}/predictions.dat", sep="\t", index=False)

    labels_map = {str(enc): cls for cls, enc in zip(le.classes_, le.transform(le.classes_))}
    with open(f"{output_dir}/labels_map.json", "w") as f:
        json.dump(labels_map, f)

    print("✓ QML training completato (struttura path CNN).")


Epoch 1/5
160/160 - 31s - loss: 3.6249 - accuracy: 0.0535 - val_loss: 3.5591 - val_accuracy: 0.0585 - 31s/epoch - 196ms/step
Epoch 2/5
160/160 - 31s - loss: 3.5100 - accuracy: 0.0945 - val_loss: 3.4671 - val_accuracy: 0.0995 - 31s/epoch - 194ms/step
Epoch 3/5
160/160 - 31s - loss: 3.4425 - accuracy: 0.0981 - val_loss: 3.4203 - val_accuracy: 0.0980 - 31s/epoch - 194ms/step
Epoch 4/5
160/160 - 31s - loss: 3.4068 - accuracy: 0.0972 - val_loss: 3.3912 - val_accuracy: 0.0945 - 31s/epoch - 196ms/step
Epoch 5/5
160/160 - 32s - loss: 3.3756 - accuracy: 0.1059 - val_loss: 3.3574 - val_accuracy: 0.1170 - 32s/epoch - 200ms/step
607/607 [==============================] - 41s 67ms/step
✓ QML training completato (struttura path CNN).
